In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy

In [4]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, ConfusionMatrixDisplay, confusion_matrix
from torchvision import datasets, transforms

In [5]:
train = datasets.MNIST('../data', train=True, transform=transforms.Compose([transforms.ToTensor()]))
test = datasets.MNIST('../data', train=False, transform=transforms.Compose([transforms.ToTensor()]))

In [6]:
def plot(X):
    img = (np.array(X.detach().cpu(), dtype='float')).reshape(28, 28)
    plt.imshow(img, cmap='gray')
    plt.show()

In [8]:
x = (train.data.float() / 255).reshape(train.data.shape[0], -1)
y = train.targets

In [9]:
input_size = x.shape[-1]
output_size = int(max(y)) + 1

In [10]:
ratios = [0.8, 0.2]
train_cnt = int(x.shape[0] * ratios[0])
valid_cnt = x.shape[0] - train_cnt
test_cnt = len(test.data)
cnts = [train_cnt, valid_cnt]
cnts, test_cnt

([48000, 12000], 10000)

In [11]:
indices = torch.randperm(x.shape[0])
x = torch.index_select(x, dim=0, index=indices)
y = torch.index_select(y, dim=0, index=indices)
x = list(x.split(cnts, dim=0))
y = list(y.split(cnts, dim=0))
x += [(test.data.float() / 255).reshape(test_cnt, -1)]
y += [test.targets]
for x_i, y_i in zip(x, y):
    print(x_i.shape, y_i.shape)

torch.Size([48000, 784]) torch.Size([48000])
torch.Size([12000, 784]) torch.Size([12000])
torch.Size([10000, 784]) torch.Size([10000])
